### 通过重定向/管道/文件接受输入

In [ ]:
# 重定向文件
#!/usr/bin/env python3
import fileinput

with fileinput.input() as f_input:
    for line in f_input:
        print(line, end='')
        
$ ls | ./filein.py          # Prints a directory listing to stdout.
$ ./filein.py /etc/passwd   # Reads /etc/passwd to stdout.
$ ./filein.py < /etc/passwd # Reads /etc/passwd to stdout.

In [ ]:
import fileinput
with fileinput.input('/etc/passwd') as f:
    for line in f:
        print(f.filename(), f.lineno(), line, end='')

### 终止程序并给出错误信息

In [ ]:
raise SystemExit('It failed!')

### 解析命令行选项

argparse 模块可被用来解析命令行选项

In [ ]:
# search.py
'''
Hypothetical command-line tool for searching a collection of
files for one or more text patterns.
'''
import argparse
parser = argparse.ArgumentParser(description='Search some files')

# action 参数指定跟属性对应的处理逻辑
# 通常的值为 store ,被用来存储某个值或将多个参数值收集到一个列表中
parser.add_argument(dest='filenames', metavar='filename', nargs='*')

# 下面的参数说明允许某个参数重复出现多次，并将它们追加到一个列表中去
parser.add_argument('-p', '--pat', metavar='pattern', required=True,
                    dest='patterns', action='append',
                    help='text pattern to search for')

# 下面的参数根据参数是否存在来设置一个 Boolean 标志
parser.add_argument('-v', dest='verbose', action='store_true',
                    help='verbose mode')

# 下面的参数接受一个单独值并将其存储为一个字符串
parser.add_argument('-o', dest='outfile', action='store',
                    help='output file')

# 下面的参数说明接受一个值，但是会将其和可能的选择值做比较，以检测其合法性
parser.add_argument('--speed', dest='speed', action='store',
                    choices={'slow','fast'}, default='slow',
                    help='search speed')

args = parser.parse_args()

# Output the collected arguments
print(args.filenames)
print(args.patterns)
print(args.verbose)
print(args.outfile)
print(args.speed)

# bash % python3 search.py -h
# usage: search.py [-h] [-p pattern] [-v] [-o OUTFILE] [--speed {slow,fast}]
#                  [filename [filename ...]]

# Search some files

# positional arguments:
#   filename

# optional arguments:
#   -h, --help            show this help message and exit
#   -p pattern, --pat pattern
#                         text pattern to search for
#   -v                    verbose mode
#   -o OUTFILE            output file
#   --speed {slow,fast}   search speed

### 运行时弹出密码输入提示

In [ ]:
import getpass

user = input('Enter your username: ')
# user = getpass.getuser()
passwd = getpass.getpass()

def svc_login(user, passwd):
    return True

if svc_login(user, passwd):    # You must write svc_login()
   print('Yay!')
else:
   print('Boo!')

### 获取终端的大小

In [9]:
import os
os.get_terminal_size()

os.terminal_size(columns=133, lines=28)

### 复制或者移动文件和目录

In [ ]:
import shutil

# Copy src to dst. (cp src dst)
shutil.copy(src, dst)

# Copy files, but preserve metadata (cp -p src dst)
shutil.copy2(src, dst)

# Copy directory tree (cp -R src dst)
shutil.copytree(src, dst)

# Move src to dst (mv src dst)
shutil.move(src, dst)

In [ ]:
# 默认情况下，对于符号链接而已这些命令处理的是它指向的东西
# 想保留被复制目录中的符号链接
shutil.copytree(src, dst, symlinks=True)

# 忽略某种模式的文件名
shutil.copytree(src, dst, ignore=shutil.ignore_patterns('*~', '*.pyc'))

# 打包为一个单独的异常，到了最后再抛出
try:
    shutil.copytree(src, dst)
except shutil.Error as e:
    for src, dst, msg in e.args[0]:
         # src is source name
         # dst is destination name
         # msg is error message from exception
         print(dst, src, msg)

### 创建和解压归档文件

In [ ]:
import shutil
shutil.unpack_archive('Python-3.3.0.tgz')

shutil.make_archive('py33','zip','Python-3.3.0')

### 通过文件名查找文件

os.walk() 方法为我们遍历目录树， 

每次进入一个目录，它会返回一个三元组，

包含相对于查找目录的相对路径，一个该目录下的目录名列表， 以及那个目录下面的文件名列表

In [ ]:
#!/usr/bin/env python3.3
import os

def findfile(start, name):
    for relpath, dirs, files in os.walk(start):
        # 对于每个元组，只需检测一下目标文件名是否在文件列表中
        if name in files:
            # 如果是就使用 os.path.join() 合并路径
            full_path = os.path.join(start, relpath, name)
            # 修正结果
            print(os.path.normpath(os.path.abspath(full_path)))

if __name__ == '__main__':
    findfile(sys.argv[1], sys.argv[2])

### 读取配置文件

```
; config.ini
; Sample configuration file

[installation]
library=%(prefix)s/lib
include=%(prefix)s/include
bin=%(prefix)s/bin
prefix=/usr/local

# Setting related to debug configuration
[debug]
log_errors=true
show_warnings=False

[server]
port: 8080
nworkers: 32
pid-file=/tmp/spam.pid
root=/www/root
signature:
    =================================
    Brought to you by the Python Cookbook
    =================================
```

In [ ]:
from configparser import ConfigParser
cfg = ConfigParser()
cfg.read('config.ini')
# ['config.ini']
cfg.sections()
# ['installation', 'debug', 'server']
cfg.get('installation','library')
# '/usr/local/lib'
cfg.getboolean('debug','log_errors')
# True
cfg.getint('server','port')
# 8080
cfg.getint('server','nworkers')
# 32
print(cfg.get('server','signature'))
# \=================================
# Brought to you by the Python Cookbook
# \=================================

In [ ]:
cfg.set('server','port','9000')
cfg.set('debug','log_errors','False')
import sys
cfg.write(sys.stdout)

### 给简单脚本增加日志功能

In [20]:
import logging

def main():
    # Configure the logging system
    logging.basicConfig(
        filename='app.log',
        level=logging.ERROR
    )
    
    logging.basicConfig(
         filename='app.log',
         level=logging.WARNING,
         format='%(levelname)s:%(asctime)s:%(message)s'
    )

    # Variables (to make the calls that follow work)
    hostname = 'www.python.org'
    item = 'spam'
    filename = 'data.csv'
    mode = 'r'

    # Example logging calls (insert into your program)
    logging.critical('Host %s unknown', hostname)
    logging.error("Couldn't find %r", item)
    logging.warning('Feature is deprecated')
    logging.info('Opening file %r, mode=%r', filename, mode)
    logging.debug('Got here')

if __name__ == '__main__':
    main()

In [ ]:
# 写到标准错误中
logging.basicConfig(level=logging.INFO)
# 直接修改它
logging.getLogger().level = logging.DEBUG

### 给函数库增加日志功能

In [24]:
# somelib.py

import logging
log = logging.getLogger(__name__)
log.addHandler(logging.NullHandler())

# Example function (for testing)
def func():
    log.critical('A Critical Error!')
    log.debug('A debug message')
    
import logging
logging.basicConfig()
func()

### 实现一个计时器

In [28]:
import time

class Timer:
    def __init__(self, func=time.perf_counter):
        self.elapsed = 0.0
        self._func = func
        self._start = None

    def start(self):
        if self._start is not None:
            raise RuntimeError('Already started')
        self._start = self._func()

    def stop(self):
        if self._start is None:
            raise RuntimeError('Not started')
        end = self._func()
        self.elapsed += end - self._start
        self._start = None

    def reset(self):
        self.elapsed = 0.0

    @property
    def running(self):
        return self._start is not None

    def __enter__(self):
        self.start()
        return self

    def __exit__(self, *args):
        self.stop()
        
def countdown(n):
    while n > 0:
        n -= 1

# Use 1: Explicit start/stop
t = Timer()
t.start()
countdown(1000000)
t.stop()
print(t.elapsed)

# Use 2: As a context manager
with t:
    countdown(1000000)

print(t.elapsed)

with Timer() as t2:
    countdown(1000000)
print(t2.elapsed)

0.05640609800002494
0.10481152300008034
0.04572730500012767


### 限制内存和CPU的使用量

In [ ]:
import signal
import resource
import os

# 限制 cpu 时间
def time_exceeded(signo, frame):
    print("Time's up!")
    raise SystemExit(1)

def set_max_runtime(seconds):
    # Install the signal handler and set a resource limit
    soft, hard = resource.getrlimit(resource.RLIMIT_CPU)
    resource.setrlimit(resource.RLIMIT_CPU, (seconds, hard))
    signal.signal(signal.SIGXCPU, time_exceeded)

if __name__ == '__main__':
    set_max_runtime(15)
    while True:
        pass

In [ ]:
import resource

# 限制内存
def limit_memory(maxsize):
    soft, hard = resource.getrlimit(resource.RLIMIT_AS)
    resource.setrlimit(resource.RLIMIT_AS, (maxsize, hard))

### 启动一个WEB浏览器

In [29]:
import webbrowser
webbrowser.open('http://www.python.org')

True